In [1]:
import pandas as pd
import json
from collections import defaultdict
import re
import dataExtraction2
import dataExtraction3
import dataExtraction4

#### Con este bucle obtenemos los datos sobre la inicialización de todos los niveles jugados. PRUEBA

Si se han usado los pasos minimos:

In [2]:
def porDefecto():
    return defaultdict(lambda: {"tarjetasCreadas" : 0, "tarjetasMovidas" : 0, "tarjetasEliminadas" : 0})

def extraerDatosTarjetas(rawData):
    erGameObject = re.compile(r'\bgame-object$\b')
    erInteracted = re.compile(r'\binteracted$\b')
    
    data = defaultdict(porDefecto)

    for evento in rawData:
        
        verb = evento["verb"]["id"]
        obj = evento["object"]["definition"]["type"]
        if erGameObject.search(obj) : #Si el objeto de la acción es un objecto
            name = evento["actor"]["name"]

            if erInteracted.search(verb):
                print(evento)
                try:
                    levelCode = evento["result"]["extensions"]["level"]
                    dataExtraction2.anadirAlDiccionario(data, evento, name, levelCode)
                except KeyError:
                    pass

    return data            

In [3]:
#Laseres
def porDefecto():
    return defaultdict(lambda: {"laserCreados" : 0, "laserMovidos" : 0, "laserEliminados" : 0})

def extraerDatosLaseres(rawData):
    erGameObject = re.compile(r'\bgame-object$\b')
    erInteracted = re.compile(r'\binteracted$\b')
    
    data = defaultdict(porDefecto)

    for evento in rawData:
        
        verb = evento["verb"]["id"]
        obj = evento["object"]["definition"]["type"]
        if erGameObject.search(obj) : #Si el objeto de la acción es un objecto
            name = evento["actor"]["name"]

            if erInteracted.search(verb):
                print(evento)
                try:
                    dataExtraction3.anadirAlDiccionario(data, evento, name)
                except KeyError:
                    pass

    return data 

In [4]:
#Retry button
def porDefecto():
    return defaultdict(lambda: {"botonRetryUsado" : 0})

def extraerDatosBoton(rawData):
    erGameObject = re.compile(r'\bgame-object$\b')
    erInteracted = re.compile(r'\binteracted$\b')
    erButton = re.compile(r'\bretry_button$\b')
    
    data = defaultdict(porDefecto)

    for evento in rawData:
        
        verb = evento["verb"]["id"]
        obj = evento["object"]["definition"]["type"]
        if erGameObject.search(obj) : #Si el objeto de la acción es un objecto
            name = evento["actor"]["name"]

            if erInteracted.search(verb):
                if erButton.search(evento["object"]["id"]):
                    #print(evento)
                    try:
                        dataExtraction4.anadirAlDiccionario(data, evento, name)
                    except KeyError:
                        pass

    return data 

In [5]:
"""import pandas as pd
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
import json
from collections import defaultdict
import re
import dataExtraction2

def extraerDatosTarjetas(rawData):
    erGameObject = re.compile(r'\bgame-object$\b')
    erInteracted = re.compile(r'\binteracted$\b')
    
    data = defaultdict(porDefecto)

    for evento in rawData:
        verb = evento["verb"]["id"]
        obj = evento["object"]["definition"]["type"]
        if erGameObject.search(obj) : #Si el objeto de la acción es un objecto
            name = evento["actor"]["name"]

            if erInteracted.search(verb):
                try:
                    levelCode = evento["result"]["extensions"]["level"]
                    dataExtraction2.anadirAlDiccionario(data, evento, name, levelCode)
                except KeyError:
                    pass

    return data            

%matplotlib widget

JSONFile = open('traces_Articoding_Escolapias.json')
rawData = json.load(JSONFile)
data = extraerDatosTarjetas(rawData)

# Creamos o entorno
fig = plt.figure()  # Creamos unha figura (contenedor para elementos a debuxar)
ax = fig.add_subplot(projection='3d')  # Engadimos un Axes á figura (contén os elementos do debuxo)
plt.xlabel("attack")  # Nombramos os eixos
plt.ylabel("hp")

# Debuxamos o scatter
ax.scatter(data['gqoj'][0]["tarjetasCreadas"], data['gqoj'][0]["tarjetasMovidas"], data['gqoj'][0]["tarjetasEliminadas"])

plt.show()"""

'import pandas as pd\nimport matplotlib as mpl\nfrom matplotlib import cm\nimport matplotlib.pyplot as plt\nfrom mpl_toolkits import mplot3d\nfrom mpl_toolkits.mplot3d import Axes3D\nimport json\nfrom collections import defaultdict\nimport re\nimport dataExtraction2\n\ndef extraerDatosTarjetas(rawData):\n    erGameObject = re.compile(r\'\x08game-object$\x08\')\n    erInteracted = re.compile(r\'\x08interacted$\x08\')\n    \n    data = defaultdict(porDefecto)\n\n    for evento in rawData:\n        verb = evento["verb"]["id"]\n        obj = evento["object"]["definition"]["type"]\n        if erGameObject.search(obj) : #Si el objeto de la acción es un objecto\n            name = evento["actor"]["name"]\n\n            if erInteracted.search(verb):\n                try:\n                    levelCode = evento["result"]["extensions"]["level"]\n                    dataExtraction2.anadirAlDiccionario(data, evento, name, levelCode)\n                except KeyError:\n                    pass\n\n  

In [6]:
#%%
"""pokemon = pd.read_csv('pokemon.csv',  # Cargamos o csv
                       delimiter=',',
                       header=0)
pokemon.drop(['abilities','against_bug', 'against_dark', 'against_dragon', 'against_electric', 'against_fairy', 'against_fight', 'against_fire', 'against_flying', 'against_ghost', 'against_grass', 'against_ground', 'against_ice', 'against_normal', 'against_poison', 'against_psychic', 'against_rock', 'against_steel', 'against_water', 'base_egg_steps', 'base_happiness', 'base_total', 'capture_rate', 'classfication', 'experience_growth', 'height_m', 'japanese_name', 'name', 'percentage_male', 'pokedex_number', 'type1', 'type2', 'weight_kg', 'generation', 'sp_attack', 'sp_defense', 'defense', 'is_legendary'], axis = 'columns', inplace=True)

%matplotlib widget

# Creamos o entorno
fig = plt.figure()  # Creamos unha figura (contenedor para elementos a debuxar)
ax = fig.add_subplot(projection='3d')  # Engadimos un Axes á figura (contén os elementos do debuxo)
plt.xlabel("attack")  # Nombramos os eixos
plt.ylabel("hp")

# Debuxamos o scatter
ax.scatter(pokemon['attack'], pokemon['hp'], pokemon['speed'])

plt.show()"""


'pokemon = pd.read_csv(\'pokemon.csv\',  # Cargamos o csv\n                       delimiter=\',\',\n                       header=0)\npokemon.drop([\'abilities\',\'against_bug\', \'against_dark\', \'against_dragon\', \'against_electric\', \'against_fairy\', \'against_fight\', \'against_fire\', \'against_flying\', \'against_ghost\', \'against_grass\', \'against_ground\', \'against_ice\', \'against_normal\', \'against_poison\', \'against_psychic\', \'against_rock\', \'against_steel\', \'against_water\', \'base_egg_steps\', \'base_happiness\', \'base_total\', \'capture_rate\', \'classfication\', \'experience_growth\', \'height_m\', \'japanese_name\', \'name\', \'percentage_male\', \'pokedex_number\', \'type1\', \'type2\', \'weight_kg\', \'generation\', \'sp_attack\', \'sp_defense\', \'defense\', \'is_legendary\'], axis = \'columns\', inplace=True)\n\n%matplotlib widget\n\n# Creamos o entorno\nfig = plt.figure()  # Creamos unha figura (contenedor para elementos a debuxar)\nax = fig.add_sub

In [7]:
JSONFile = open('traces_Articoding_Escolapias.json')
rawData = json.load(JSONFile)
#data = extraerDatosTarjetas(rawData)
#data = extraerDatosLaseres(rawData)
data = extraerDatosBoton(rawData)
pd.DataFrame(data)

JSONFile.close()

pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)
#pd.DataFrame(data)

pd.DataFrame(data)

{'actor': {'name': 'gqoj', 'account': {'homePage': 'https://simva-api.simva.e-ucm.es', 'username': 'gqoj'}}, 'verb': {'id': 'http://adlnet.gov/expapi/verbs/interacted'}, '_id': '628179b5a6ba41003d3bada2', 'object': {'definition': {'type': 'https://w3id.org/xapi/seriousgames/activity-types/game-object'}, 'id': 'https://simva-api.simva.e-ucm.es/activities/628179b5a6ba41003d3bada2/retry_button'}, 'timestamp': '2022-05-16T10:37:39.640Z'}
{'actor': {'name': 'uala', 'account': {'homePage': 'https://simva-api.simva.e-ucm.es', 'username': 'uala'}}, 'verb': {'id': 'http://adlnet.gov/expapi/verbs/interacted'}, '_id': '628179b5a6ba41003d3bada2', 'object': {'definition': {'type': 'https://w3id.org/xapi/seriousgames/activity-types/game-object'}, 'id': 'https://simva-api.simva.e-ucm.es/activities/628179b5a6ba41003d3bada2/retry_button'}, 'timestamp': '2022-05-17T07:41:19.514Z'}
{'actor': {'name': 'hjpbw', 'account': {'homePage': 'https://simva-api.simva.e-ucm.es', 'username': 'hjpbw'}}, 'verb': {'id'

,gqoj,uala,hjpbw,ftpnb,loemd,gcpze,ikcar,fkmtp,sxipu,vsxld,ipdcp,qeqyj,khadt,gleqe,bdwyj,ejeuw,srjon,zwzfv,uzzpo,epqkm,zxvkj,eprdu,pbgky,kmqou,xnpqf,rdiaf,grkij,ubmtx,auvqa,yvpmf,onyep,bhyhw,edxmq,asopk,pygoc,oahey,yfohf,tekwp,accul,zcibf,nmveq,tjfzy,swvxx,dwqwi,dunul,sjpkc,krahz,cmbfi,kkoif,zbgwk,psido,gdljy,mymho,tinco,wjaxi,dvmlu,gguuc,fivin,kkliq,tgwmn,yxewz,ewqbs,ucdvy,lwryl,fyyya,rziia,dmwvy,ztyed,yxjji,mjiso,camhi,hqmhq,hpdxj,vddlg,stiry,ngavz,bjvnt,tqgyp,huyyb,wqmgq,yazod,vvqlm,atlzs,aotlm,tavin,tvxow,ahxym,rqlug,idmzi,xtdrq,jozyx,knewj,mgaak,zdnso,qfsdj,jhbuv,lhrbj,buvgs,icllt,thaho,rqaby,mewpz,qwnpf,khfha,ptmkr,esrit,yyvas,pbwkf,gzgfk,sbexy,nhbnj,uniee,rkmrw,nuasx,fmtrq,pzfdd,hcbyl,itkas,nplrf,kzles,lohyo,rfofs,cojqb,qbsjv,ixrux,eayrh,qgcgc,orjyn,tcvyz,rsjxu,blvkx,pleui,xbmmw,fmupf,tflzv,sybwc,vkfnt,bajbm,raqdt,prpej,pqwaf,olugf,fbldj,epjcx
0,{'botonRetryUsado': 5},{'botonRetryUsado': 1},{'botonRetryUsado': 35},{'botonRetryUsado': 20},{'botonRetryUsado': 14},{'botonRetryUsado': 13},{'botonRetryUsado': 18},{'botonRetryUsado': 42},{'botonRetryUsado': 72},{'botonRetryUsado': 12},{'botonRetryUsado': 29},{'botonRetryUsado': 30},{'botonRetryUsado': 26},{'botonRetryUsado': 20},{'botonRetryUsado': 36},{'botonRetryUsado': 26},{'botonRetryUsado': 9},{'botonRetryUsado': 46},{'botonRetryUsado': 17},{'botonRetryUsado': 15},{'botonRetryUsado': 28},{'botonRetryUsado': 24},{'botonRetryUsado': 31},{'botonRetryUsado': 39},{'botonRetryUsado': 5},{'botonRetryUsado': 72},{'botonRetryUsado': 32},{'botonRetryUsado': 64},{'botonRetryUsado': 22},{'botonRetryUsado': 73},{'botonRetryUsado': 72},{'botonRetryUsado': 27},{'botonRetryUsado': 78},{'botonRetryUsado': 31},{'botonRetryUsado': 15},{'botonRetryUsado': 19},{'botonRetryUsado': 37},{'botonRetryUsado': 88},{'botonRetryUsado': 54},{'botonRetryUsado': 16},{'botonRetryUsado': 97},{'botonRetryUsado': 92},{'botonRetryUsado': 55},{'botonRetryUsado': 24},{'botonRetryUsado': 16},{'botonRetryUsado': 134},{'botonRetryUsado': 14},{'botonRetryUsado': 14},{'botonRetryUsado': 21},{'botonRetryUsado': 58},{'botonRetryUsado': 34},{'botonRetryUsado': 51},{'botonRetryUsado': 44},{'botonRetryUsado': 38},{'botonRetryUsado': 25},{'botonRetryUsado': 120},{'botonRetryUsado': 66},{'botonRetryUsado': 21},{'botonRetryUsado': 51},{'botonRetryUsado': 109},{'botonRetryUsado': 57},{'botonRetryUsado': 45},{'botonRetryUsado': 63},{'botonRetryUsado': 32},{'botonRetryUsado': 15},{'botonRetryUsado': 128},{'botonRetryUsado': 72},{'botonRetryUsado': 68},{'botonRetryUsado': 60},{'botonRetryUsado': 13},{'botonRetryUsado': 21},{'botonRetryUsado': 21},{'botonRetryUsado': 44},{'botonRetryUsado': 10},{'botonRetryUsado': 7},{'botonRetryUsado': 49},{'botonRetryUsado': 18},{'botonRetryUsado': 115},{'botonRetryUsado': 41},{'botonRetryUsado': 96},{'botonRetryUsado': 28},{'botonRetryUsado': 45},{'botonRetryUsado': 41},{'botonRetryUsado': 25},{'botonRetryUsado': 38},{'botonRetryUsado': 22},{'botonRetryUsado': 29},{'botonRetryUsado': 78},{'botonRetryUsado': 34},{'botonRetryUsado': 21},{'botonRetryUsado': 40},{'botonRetryUsado': 12},{'botonRetryUsado': 14},{'botonRetryUsado': 26},{'botonRetryUsado': 7},{'botonRetryUsado': 29},{'botonRetryUsado': 23},{'botonRetryUsado': 35},{'botonRetryUsado': 27},{'botonRetryUsado': 54},{'botonRetryUsado': 43},{'botonRetryUsado': 58},{'botonRetryUsado': 32},{'botonRetryUsado': 65},{'botonRetryUsado': 65},{'botonRetryUsado': 60},{'botonRetryUsado': 67},{'botonRetryUsado': 37},{'botonRetryUsado': 40},{'botonRetryUsado': 45},{'botonRetryUsado': 76},{'botonRetryUsado': 48},{'botonRetryUsado': 20},{'botonRetryUsado': 54},{'botonRetryUsado': 20},{'botonRetryUsado': 27},{'botonRetryUsado': 43},{'botonRetryUsado': 70},{'botonRetryUsado': 27},{'botonRetryUsado': 10},{'botonRetryUsado': 74},{'botonRetryUsado': 84},{'botonRetryUsado': 39},{'botonRetryUsado': 55},{'botonRetryUsado': 52},{'botonRetryUsado': 39},{'botonRetryUsado': 36},{'botonRetryUsado': 75},{'botonRetryUsado': 31},{'botonRetryUsado': 99},{'botonRetryUsad